# utilisant l'algorithme génétique pour optimiser un modèle de réseau de neurones pour prédire le diabète à partir du Diabetes Dataset:

In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from deap import base, creator, tools, algorithms

In [2]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from deap import base, creator, tools, algorithms

# Chargement des données
df = pd.read_csv('diabetes.csv')
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

# Normalisation des données
sc = StandardScaler()
X = sc.fit_transform(X)

# Séparation des données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Fonction d'évaluation pour le modèle de réseau de neurones
def evaluate_nn(individual):
    model = Sequential()
    model.add(Dense(individual[0], input_dim=X_train.shape[1], activation='relu'))
    for i in range(1, len(individual)):
        model.add(Dense(individual[i], activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    opt = Adam(learning_rate=0.01)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

    model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)
    score = model.evaluate(X_test, y_test, verbose=0)

    return score[1],

# Définition des paramètres de l'algorithme génétique avec DEAP
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)
toolbox = base.Toolbox()
toolbox.register("attr_int", np.random.randint, 2, 16)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_int, n=3)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evaluate_nn)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutUniformInt, low=2, up=16, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)

# Exécution de l'algorithme génétique avec DEAP
pop = toolbox.population(n=10)
algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=10, verbose=False)

# Récupération du meilleur individu
best_ind = tools.selBest(pop, k=1)[0]
print("Meilleur chromosome: ", best_ind)

# Entraînement et évaluation du modèle final avec le meilleur individu
model = Sequential()
model.add(Dense(best_ind[0], input_dim=X_train.shape[1], activation='relu'))
for i in range(1, len(best_ind)):
    model.add(Dense(best_ind[i], activation='relu'))
model.add(Dense(1, activation='sigmoid'))

opt = Adam(learning_rate=0.01)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)
score = model.evaluate(X_test, y_test, verbose=0)

print("Perte sur l'ensemble de test: ", score[0])
print("Exactitude sur l'ensemble de test: ", score[1])


Meilleur chromosome:  [6, 2, 8]
Perte sur l'ensemble de test:  0.5797340273857117
Exactitude sur l'ensemble de test:  0.7597402334213257


# Affiche le Meilleur fitness

In [5]:
best_fitness = evaluate_nn(best_ind)[0]
print("Meilleur fitness : ", best_fitness)

Meilleur fitness :  0.8051947951316833
